In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gurobipy import Model, GRB, quicksum

In [2]:
result_moo = pd.read_csv('./output_VM/3_gap_0.025/result_5_MOO_2.csv')
result_moo.head()

,employee,company,assigned_task,sum_sp,wasted_sp,assessment_score
0,Talent 1,['P3'],['T270'],8,12,[-0.027166504381694268]
1,Talent 2,"['P3', 'P3']","['T97', 'T166']",11,9,"[-0.017692307692307688, -0.017582417582417582]"
2,Talent 3,"['P2', 'P2']","['T201', 'T297']",11,9,"[-0.01773854560739807, -0.011648351648351648]"
3,Talent 4,"['P5', 'P5']","['T153', 'T296']",10,10,"[-0.01749788672865596, -0.011564986737400529]"
4,Talent 5,['P4'],['T39'],8,12,[-0.029608127721335267]


In [3]:
salary_job_df = pd.read_csv('./data/linreg_salary_job.csv')
salary_job_df.head()

,job_role,junior,middle,senior
0,Artificial intelligence,"15,359,986","20,517,907","34,001,747"
1,Data Analyst,"10,361,232","15,519,153","29,002,993"
2,Data Engineer,"13,996,396","19,154,317","32,638,157"
3,Data Scientist,"16,557,921","21,715,842","35,199,682"


In [7]:
# make list of job roles
job_roles = salary_job_df['job_role'].unique()
job_roles

array(['Artificial intelligence', 'Data Analyst', 'Data Engineer',
       'Data Scientist'], dtype=object)

In [6]:
# Create a dictionary of levels
levels = {}

# iterate each column from second column
for col in salary_job_df.columns[1:]:
    levels[col] = salary_job_df[col].unique()

levels

{'junior': array(['15,359,986', '10,361,232', '13,996,396', '16,557,921'],
       dtype=object),
 'middle': array(['20,517,907', '15,519,153', '19,154,317', '21,715,842'],
       dtype=object),
 'senior': array(['34,001,747', '29,002,993', '32,638,157', '35,199,682'],
       dtype=object)}

In [ ]:
salary_job_df = pd.read_csv('./data/linreg_salary_job.csv')
salary_job_df.head()

,job_role,junior,middle,senior
0,Artificial intelligence,"15,359,986","20,517,907","34,001,747"
1,Data Analyst,"10,361,232","15,519,153","29,002,993"
2,Data Engineer,"13,996,396","19,154,317","32,638,157"
3,Data Scientist,"16,557,921","21,715,842","35,199,682"


In [31]:
role_levels_salary = {
    ('Artificial Intelligence', 'Junior'): 15359986,
    ('Artificial Intelligence', 'Middle'): 20517907,
    ('Artificial Intelligence', 'Senior'): 34001747,
    ('Data Analyst', 'Junior'): 10361232,
    ('Data Analyst', 'Middle'): 15519153,
    ('Data Analyst', 'Senior'): 29002993,
    ('Data Engineer', 'Junior'): 13996396,
    ('Data Engineer', 'Middle'): 19154317,
    ('Data Engineer', 'Senior'): 32638157,
    ('Data Scientist', 'Junior'): 16557921,
    ('Data Scientist', 'Middle'): 21715842,
    ('Data Scientist', 'Senior'): 35199682,
}
role_levels_salary

{('Artificial Intelligence', 'Junior'): 15359986,
 ('Artificial Intelligence', 'Middle'): 20517907,
 ('Artificial Intelligence', 'Senior'): 34001747,
 ('Data Analyst', 'Junior'): 10361232,
 ('Data Analyst', 'Middle'): 15519153,
 ('Data Analyst', 'Senior'): 29002993,
 ('Data Engineer', 'Junior'): 13996396,
 ('Data Engineer', 'Middle'): 19154317,
 ('Data Engineer', 'Senior'): 32638157,
 ('Data Scientist', 'Junior'): 16557921,
 ('Data Scientist', 'Middle'): 21715842,
 ('Data Scientist', 'Senior'): 35199682}

In [37]:
# Parameters
role = ['Artificial Intelligence', 'Data Analyst', 'Data Engineer', 'Data Scientist']
level = ['Junior', 'Middle', 'Senior']

curr_workforce = {
    ('Artificial Intelligence', 'Junior'): 1000,
    ('Artificial Intelligence', 'Middle'): 1440,
    ('Artificial Intelligence', 'Senior'): 1650,
    ('Data Analyst', 'Junior'): 500,
    ('Data Analyst', 'Middle'): 700,
    ('Data Analyst', 'Senior'): 1000,
    ('Data Engineer', 'Junior'): 900,
    ('Data Engineer', 'Middle'): 1100,
    ('Data Engineer', 'Senior'): 1520,
    ('Data Scientist', 'Junior'): 600,
    ('Data Scientist', 'Middle'): 800,
    ('Data Scientist', 'Senior'): 1200,
}

In [8]:
# create a model
model = Model('manpower')

Restricted license - for non-production use only - expires 2025-11-24


In [41]:
# define the decision variables redundancy if laid off is 1, else 0
redundancy = model.addVars(role, level, vtype=GRB.BINARY, name='redundancy')
redundancy

{('Artificial Intelligence', 'Junior'): <gurobi.Var *Awaiting Model Update*>,
 ('Artificial Intelligence', 'Middle'): <gurobi.Var *Awaiting Model Update*>,
 ('Artificial Intelligence', 'Senior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Analyst', 'Junior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Analyst', 'Middle'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Analyst', 'Senior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Engineer', 'Junior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Engineer', 'Middle'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Engineer', 'Senior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Scientist', 'Junior'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Scientist', 'Middle'): <gurobi.Var *Awaiting Model Update*>,
 ('Data Scientist', 'Senior'): <gurobi.Var *Awaiting Model Update*>}

In [15]:
# define the objective function: minimize total employee layoffs
model.setObjective(quicksum(redundancy[i, j] for i in job_roles for j in levels.keys()) * cost_redundancy, GRB.MINIMIZE)

In [16]:
# define the constraints: workforce balance
for i in job_roles:
    for j in levels.keys():
        model.addConstr(x[i, j] - redundancy[i, j] >= 0, name=f'workforce_balance_{i}_{j}')

In [17]:
# define the constraints: total employees
total_employees = 112
model.addConstr(quicksum(x[i, j] for i in job_roles for j in levels.keys()) <= total_employees, name='total_employees')

<gurobi.Constr *Awaiting Model Update*>

In [21]:
# Print the results
if model.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    for j in job_roles:
        for l in levels:
            if redundancy[j, l].x > 0:
                print(f'{j} {l}:')
                print(f'  Employees: {x[j, l].x}')
                print(f'  Redundancy: {redundancy[j, l].x}')
else:
    print('No optimal solution found.')

Optimal solution found:
